In [1]:
clients = [
    {'account': 'saiful.aromax@gmail.com', 'devices': #Client-1
        [
            {'device_id': 'bfc6fdb8a9d07ef71cyuud', 'type': 'hub'},
            {'device_id': 'bf7adcca218736d1beltu0', 'type': 'TRV'},
            {'device_id': 'bfc8da7daa7a40e10dgtgk', 'type': 'socket'}
        ]
    },
    {'account': 'fstatazizi@gmail.com', 'devices': #Client-2
        [
            {'device_id': 'bf0867de0cbdec04744czw', 'type': 'hub'},
            {'device_id': 'bf7356030aa2552ff4akq1', 'type': 'thermostat'},
            {'device_id': 'bf247826822493b7052aya', 'type': 'TRV'},
            {'device_id': 'bfe22383bb69caf809simm', 'type': 'TRV'}
        ]
    }
]

In [3]:
for client in clients:
    print(client['account'])
    for device in client['devices']:
        print(f'Device ID: {device['device_id']} || Devide type: {device['type']}')

saiful.aromax@gmail.com
Device ID: bfc6fdb8a9d07ef71cyuud || Devide type: hub
Device ID: bf7adcca218736d1beltu0 || Devide type: TRV
Device ID: bfc8da7daa7a40e10dgtgk || Devide type: socket
fstatazizi@gmail.com
Device ID: bf0867de0cbdec04744czw || Devide type: hub
Device ID: bf7356030aa2552ff4akq1 || Devide type: thermostat
Device ID: bf247826822493b7052aya || Devide type: TRV
Device ID: bfe22383bb69caf809simm || Devide type: TRV


GET/SET Tuya Zigbee IOT Devices

In [1]:
import hashlib
import hmac
import json
import urllib
import urllib.parse
from urllib.request import urlopen, Request
from datetime import datetime
import requests

BASE_URL       = "https://openapi.tuyaeu.com"
LOGIN_URL      = "/v1.0/token?grant_type=1"
ATTRIBUTES_URL = "/v2.0/cloud/thing/{device_id}/shadow/properties"

def make_request(url, params=None, headers=None):
    if params:
        url = url + "?" + urllib.parse.urlencode(params)
    request = Request(url, headers=headers or {})

    try:
        with urlopen(request, timeout=10) as response:
            return response, response.read().decode("utf-8")

    except Exception as error:
        return error, ""

def get_timestamp(now = datetime.now()):
    return str(int(datetime.timestamp(now)*1000))

def get_sign(payload, client_secret):
    byte_key = bytes(client_secret, 'UTF-8')
    message = payload.encode()
    sign = hmac.new(byte_key, message, hashlib.sha256).hexdigest()
    return sign.upper()

def get_access_token(now, client_id, client_secret):

    timestamp = get_timestamp(now)
    string_to_sign = client_id + timestamp + "GET\n" + \
        "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855\n" + \
        "\n" + \
        LOGIN_URL
    signed_string = get_sign(string_to_sign, client_secret)

    headers = {
            "client_id": client_id,
            "sign": signed_string,
            "t": timestamp,
            "mode": "cors",
            "sign_method": "HMAC-SHA256",
            "Content-Type": "application/json"
            }

    response, body = make_request(BASE_URL + LOGIN_URL, headers = headers)

    json_result = json.loads(body)["result"]
    access_token = json_result["access_token"]
    return access_token

def get_status(device_id, client_id, client_secret):
    access_token = get_access_token(datetime.now(), client_id, client_secret)
    url = ATTRIBUTES_URL.format(device_id=device_id)
    timestamp = get_timestamp()
    string_to_sign = client_id + access_token + timestamp + "GET\n" + \
        "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855\n" + \
        "\n" + \
        url
    
    signed_string = get_sign(string_to_sign, client_secret)
    headers = {
            "client_id": client_id,
            "sign": signed_string,
            "access_token": access_token,
            "t": timestamp,
            "mode": "cors",
            "sign_method": "HMAC-SHA256",
            "Content-Type": "application/json"
            }

    response, body = make_request(BASE_URL + url, headers = headers)

    json_result = json.loads(body)
    properties = json_result["result"]["properties"]
    output = list()
    for item in properties:
        if ('value' in item) and ('code' in item):
            if (item['code'] == 'temp_set') or (item['code'] == 'temp_current') or (item['code'] == 'TempCurrent') or (item['code'] == 'TempSet') or (item['code'] == 'switch_1'):
                if (item['code'] == 'temp_current') or (item['code'] == 'TempSet') or (item['code'] == 'TempCurrent'):
                    output.append({item['code']: item['value'] / 10})
                elif (item['code'] == 'switch_1'):
                    if item['value'] == True:
                        output.append({'HeatPump': 'ON'})
                    else:
                        output.append({'HeatPump': 'OFF'})
                else:
                    output.append({item['code']: item['value']})
    return properties

def set_temperature(device_id, temperature, client_id, client_secret):
    # get_device_info = get_status(device_id)
    # is_thermostat = any('TempSet' in d for d in get_device_info)
    # if is_thermostat:
    #     temperature = temperature * 10
    access_token = get_access_token(datetime.now(), client_id, client_secret)
    temperature = str(temperature)
    sign_url = "/v1.0/devices/" + device_id + "/commands"
    body_string = '{ "commands": [ { "code": "temp_set", "value": ' + temperature + ' } ] }'
    body_string_encoded = hashlib.sha256(body_string.encode()).hexdigest()

    sign_string1 = "POST\n" + body_string_encoded + "\n\n" + sign_url
    
    timestamp = get_timestamp()

    sign_string2 = client_id + access_token + timestamp + sign_string1
    
    command_url = "https://openapi.tuyaeu.com/v1.0/devices/" + device_id + "/commands"

    signed_string = get_sign(sign_string2, client_secret)
    
    headers = {
            "client_id": client_id,
            "sign": signed_string,
            "access_token": access_token,
            "t": timestamp,
            "mode": "cors",
            "sign_method": "HMAC-SHA256",
            "Content-Type": "application/json"
            }

    data = {
        "commands": [
            {"code": "temp_set", "value": temperature}
        ]
    }

    response = requests.post(command_url, data=body_string, headers=headers)

    if response.status_code == 200:
        print('Temperature set successfully')
    else:
        print('Error setting temperature:', response.text)
        
def set_heatpump(device_id, command, client_id, client_secret):
    access_token = get_access_token(datetime.now(), client_id, client_secret)
    sign_url = "/v1.0/devices/" + device_id + "/commands"
    body_string = '{ "commands": [ { "code": "switch_1", "value": ' + command + ' } ] }'
    body_string_encoded = hashlib.sha256(body_string.encode()).hexdigest()
    sign_string1 = "POST\n" + body_string_encoded + "\n\n" + sign_url
    
    timestamp = get_timestamp()

    sign_string2 = client_id + access_token + timestamp + sign_string1
    
    command_url = "https://openapi.tuyaeu.com/v1.0/devices/" + device_id + "/commands"

    signed_string = get_sign(sign_string2, client_secret)
    
    headers = {
            "client_id": client_id,
            "sign": signed_string,
            "access_token": access_token,
            "t": timestamp,
            "mode": "cors",
            "sign_method": "HMAC-SHA256",
            "Content-Type": "application/json"
            }

    response = requests.post(command_url, data=body_string, headers=headers)
    if response.status_code == 200:
        print('Heatpump set Successfully')
    else:
        print('Error setting Heatpump:', response.text)

In [2]:
client_id = "vtj3ft7hadfea5y3yvs7"
client_secret = "16347a3b50984ae696e6887e4d503fa2"
dd = get_status('bf7356030aa2552ff4akq1', client_id, client_secret)
print(dd)

[{'code': 'Power', 'custom_name': '', 'dp_id': 1, 'time': 1715268260648, 'value': True}, {'code': 'TempSet', 'custom_name': '', 'dp_id': 2, 'time': 1715271064370, 'value': 28}, {'code': 'TempCurrent', 'custom_name': '', 'dp_id': 3, 'time': 1715270509240, 'value': 61}, {'code': 'Mode', 'custom_name': '', 'dp_id': 4, 'time': 1715268596530, 'value': '1'}, {'code': 'ECO', 'custom_name': '', 'dp_id': 5, 'time': 1715268654870, 'value': False}, {'code': 'ChildLock', 'custom_name': '', 'dp_id': 6, 'time': 1715268651714, 'value': False}, {'code': 'program', 'custom_name': '', 'dp_id': 101, 'time': 1715268236300, 'value': 'AwYsAAgeHgseHg0eABEsABYeAAYoAAgoHgsoHg0oABEoABYeAAYoAAgoHgsoHg0oABEoABYe'}, {'code': 'floorTemp', 'custom_name': '', 'dp_id': 102, 'time': 1715264248266, 'value': 0}, {'code': 'tempSwitch', 'custom_name': '', 'dp_id': 103, 'time': 1715264248334, 'value': '1'}, {'code': 'floortempFunction', 'custom_name': '', 'dp_id': 104, 'time': 1715264248285, 'value': True}]


In [3]:
set_temperature('bf7356030aa2552ff4akq1', (20 * 2), client_id, client_secret)

Temperature set successfully


Listing of all the linked devices for each clients in a python "clients" list type variable

In [5]:
clients = [
    #Client -1: Saiful
    ["bfc6fdb8a9d07ef71cyuud", #For convention, we will set 1st item to be HUB of that house/client
     'bfc8da7daa7a40e10dgtgk',
     "bf7adcca218736d1beltu0"],
    #Client -2: Sas
    [
        "bf0867de0cbdec04744czw", #For convention, we will set 1st item to be HUB of that house/client
        "bf7356030aa2552ff4akq1",
        "bf247826822493b7052aya",
        "bfe22383bb69caf809simm"
    ]
    
]

Get Device Status Client-wise

In [6]:
for i in range(len(clients)):
    print(f"Client-{i + 1}")
    for j in range(len(clients[i])):
        if j > 0:
            print(get_status(clients[i][j]))

Client-1


KeyError: 'result'

Setting temperature for each client we got from the Survery form (all room temperature/global)

In [7]:
set_temp = [24, 25] # 24 degree celsius for Client-1 & 25 degree celsius for Client-2
for i in range(len(clients)):
    for j in range(len(clients[i])):
        if j > 0:
            print(get_status(clients[i][j]))
            set_temperature(clients[i][j], set_temp[i])

KeyError: 'result'

Setting a particular room's temperature to 26 degree celsius for Client-2

In [8]:
set_temperature(clients[1][2], 26)

Temperature set successfully


Turn on Heat pump for Client-1

In [9]:
set_heatpump(clients[0][1], 'true')

Heatpump set Successfully


Turn off Heat pump for Client-1

In [10]:
set_heatpump(clients[0][1], 'false')

Heatpump set Successfully


In [2]:
from flask import Flask, request, jsonify

app = Flask(__name__)

TRVs = {
    'user1': ['device1', 'device2'],
    'user2': ['device3']
}

device_data = {
    'device1': {'temperature': 20, 'humidity': 45},
    'device2': {'temperature': 22, 'humidity': 50},
    'device3': {'temperature': 19, 'humidity': 55}
}

@app.route('/get_trv_list', methods=['GET'])
def get_trv_list():
    user_id = request.args.get('user_id')
    if user_id in TRVs:
        return jsonify(TRVs[user_id])
    else:
        return jsonify({"error": "User not found"}), 404

@app.route('/get_device_data', methods=['GET'])
def get_device_data():
    device_id = request.args.get('device_id')
    if device_id in device_data:
        return jsonify(device_data[device_id])
    else:
        return jsonify({"error": "Device not found"}), 404

@app.route('/set_temperature', methods=['POST'])
def set_temperature():
    request_data = request.json
    device_id = request_data.get('device_id')
    set_temperature = request_data.get('set_temperature')
    
    if device_id in device_data:
        device_data[device_id]['temperature'] = set_temperature
        return jsonify({"success": True})
    else:
        return jsonify({"error": "Device not found"}), 404

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (inotify)
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/aromax/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/aromax/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1042, in launch_instance
    app.initialize(argv)
  File "/home/aromax/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/home/aromax/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 689, in initialize
    self.init_sockets()
  File "/home/aromax/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 328, in init_sock

SystemExit: 1

/home/aromax/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
